In [1]:
import torch
import dqc
import dqc.xc
import dqc.utils

In [2]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a, p):
        super().__init__()
        self.a = a
        self.p = p

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a * rho ** self.p
        
    def get_edensityxc_derivative(self, densinfo, number_of_parameter):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc_derivative(densinfo.u * 2, number_of_parameter) 
                          + self.get_edensityxc_derivative(densinfo.d * 2, number_of_parameter))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            if number_of_parameter == 0: # parameter a
                return rho ** self.p
            elif number_of_parameter == 1: # parameter p
                return self.a * rho ** (self.p - 1)

In [3]:
a = torch.nn.Parameter(torch.tensor(1.0, dtype=torch.double))
p = torch.nn.Parameter(torch.tensor(4.0, dtype=torch.double))
myxc = MyLDAX(a, p)

In [4]:
mol = dqc.Mol(moldesc="N -1 0 0; N 1 0 0", basis="3-21G")
qc = dqc.KS(mol, xc=myxc).run()
ene = qc.energy()
print(ene)

The 3-21G basis for atomz 7 does not exist, but we will download it
Downloaded to /home/rolan/miniconda3/envs/noa/lib/python3.9/site-packages/dqc/api/.database/3-21g/07.gaussian94
tensor(-27.0517, dtype=torch.float64, grad_fn=<AddBackward0>)


/home/rolan/miniconda3/envs/noa/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=9.882e+04, |f|=9.798e+04 at iter 50
  warnings.warn(ConvergenceWarning(msg))


In [6]:
qc._engine.hamilton.basis.shape

torch.Size([36572, 18])

In [5]:
dm = qc._dm.detach().clone() # density matrix

Example: here the way of calculating $$\int b_i(\vec{r}) b_l(\vec{r}) b_k(\vec{r})b_j(\vec{r})d\vec{r}$$

In [6]:
a = qc._engine.hamilton.basis
#a = torch.tensor([[1, 2, 3]])
print(a.size())
b = torch.einsum("ri,rj,rk->rijk", a, a, a)
print(b.size())
c = qc._engine.hamilton.basis_dvolume
print(c.size())
# d = torch.matmul(c.transpose(-2, -1), b)
d = torch.einsum("rl,rijk->ijkl", c, b)
print(d.size())

torch.Size([36572, 18])
torch.Size([36572, 18, 18, 18])
torch.Size([36572, 18])
torch.Size([18, 18, 18, 18])


Here we calculating $$\frac{\partial V_{XC}[\rho](\vec{r_r};\;\vec{\theta})}{\partial \rho(\vec{r_r})}$$ at points $\vec{r_r}$ of grid.

In [7]:
from dqc.utils.datastruct import AtomCGTOBasis, ValGrad, SpinParam, DensityFitInfo

def get_dvxc_wrt_dro_xc(xc, densinfo):
    # densinfo.value: (*BD, nr)
    # return:
    # potentialinfo.value: (*BD, nr)
    
    # mark the densinfo components as requiring grads
    with xc._enable_grad_densinfo(densinfo):
        with torch.enable_grad():
            edensity = xc.get_edensityxc(densinfo)  # (*BD, nr)
        grad_outputs = torch.ones_like(edensity)
        grad_enabled = torch.is_grad_enabled()

        if not isinstance(densinfo, ValGrad): # polarized case
            raise NotImplementedError("polarized case is not implemented")
        else: # unpolarized case
            if xc.family == 1:  # LDA
                potinfo, = torch.autograd.grad(
                    edensity, densinfo.value, create_graph=grad_enabled,
                    grad_outputs=grad_outputs)
                print("potential info\n", potinfo)
                derivative_of_potinfo_wrt_ro, = torch.autograd.grad(
                    potinfo, densinfo.value, create_graph=grad_enabled,
                    grad_outputs=grad_outputs)
                return ValGrad(value=derivative_of_potinfo_wrt_ro)
            else: # GGA and others
                raise NotImplementedError("Default dvxc wrt dro for family %d is not implemented" % self.family)

And here we are want to calculate $$\int b_i(\vec{r}) b_l(\vec{r}) \frac{\partial V_{XC}[\rho](\vec{r_r};\;\vec{\theta})}{\partial \rho(\vec{r_r})} b_k(\vec{r})b_j(\vec{r})d\vec{r}$$

In [11]:
import xitorch as xt

def get_dvxc_wrt_dro_from_derivative_of_potinfo_wrt_ro(hamiltonian, 
                                                       derivative_of_potinfo_wrt_ro: ValGrad) -> xt.LinearOperator:
    # obtain the vxc operator from the potential information
    # potinfo.value: (*BD, nr)
    # self.basis: (nr, nao)
    # self.grad_basis: (ndim, nr, nao)
    
    # prepare the fock matrix component from vxc
    
    # TODO: do the same stuff
    #     nao = hamiltonian.basis.shape[-1]
    #     mat = torch.zeros((*derivative_of_potinfo_wrt_ro.value.shape[:-1], nao, nao), dtype=hamiltonian.dtype, device=hamiltonian.device)
    #     vb = derivative_of_potinfo_wrt_ro.value.unsqueeze(-1) * hamiltonian.basis  # (*BD, nr, nao)
    #     mat = torch.matmul(hamiltonian.basis_dvolume.transpose(-2, -1), vb)

    #     mat = hamiltonian._orthozer.convert2(mat)
    #     mat = (mat + mat.transpose(-2, -1)) * 0.5
    
    mat = torch.einsum("r,ri,rj,rk,rl->ijkl", 
                       derivative_of_potinfo_wrt_ro.value, 
                       hamiltonian.basis, 
                       hamiltonian.basis, 
                       hamiltonian.basis,
                       hamiltonian.basis_dvolume)
    
    return mat

In [9]:
def get_dvxc_wrt_dro_dm(hamiltonian, dm):
    densinfo = SpinParam.apply_fcn(lambda dm_: hamiltonian._dm2densinfo(dm_), dm)  # value: (*BD, nr)
    print("density info\n", densinfo)
    derivative_of_potinfo_wrt_ro = get_dvxc_wrt_dro_xc(hamiltonian.xc, densinfo)  # value: (*BD, nr)
    print("derivative of potential info wrt density\n", derivative_of_potinfo_wrt_ro)
    dvxc_wrt_dro = get_dvxc_wrt_dro_from_derivative_of_potinfo_wrt_ro(hamiltonian, derivative_of_potinfo_wrt_ro)
    return dvxc_wrt_dro

In [12]:
dvxc_wrt_dro = get_dvxc_wrt_dro_dm(qc._engine.hamilton, dm)
print(dvxc_wrt_dro.size())

density info
 ValGrad(value=tensor([0.0003, 0.0003, 0.0003,  ..., 0.0000, 0.0000, 0.0000],
       dtype=torch.float64), grad=None, lapl=None, kin=None)
potential info
 tensor([6.8869e-11, 6.8869e-11, 6.8869e-11,  ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00], dtype=torch.float64, grad_fn=<MulBackward0>)
derivative of potential info wrt density
 ValGrad(value=tensor([8.0012e-07, 8.0012e-07, 8.0012e-07,  ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00], dtype=torch.float64, grad_fn=<AddBackward0>), grad=None, lapl=None, kin=None)
torch.Size([18, 18, 18, 18])


It is, I guess, tensor of Couloumb term: 
$$\iint b_i(\vec{r}) \frac{b_k(\vec{r'})b_l(\vec{r'})}{|\vec{r}-\vec{r'}|} b_j(\vec{r})d\vec{r'}d\vec{r}$$

In [13]:
electronic_repulsion_tensor = qc._engine.hamilton.el_mat
electronic_repulsion_tensor.size()

torch.Size([18, 18, 18, 18])

It must have symmetry of i<->j and k<->l permutations, let's check it. We are going to calculate something like norm of remainder.

i<->j, OK

In [14]:
torch.einsum("ij->", torch.linalg.matrix_norm(electronic_repulsion_tensor - electronic_repulsion_tensor.transpose(0, 1)))

tensor(8.9071e-13, dtype=torch.float64)

i<->k, not OK

In [15]:
torch.einsum("ij->", torch.linalg.matrix_norm(electronic_repulsion_tensor - electronic_repulsion_tensor.transpose(0, 2)))

tensor(216.4966, dtype=torch.float64)

i<->l, not OK

In [16]:
torch.einsum("ij->", torch.linalg.matrix_norm(electronic_repulsion_tensor - electronic_repulsion_tensor.transpose(0, 3)))

tensor(216.4966, dtype=torch.float64)

j<->k, not OK

In [17]:
torch.einsum("ij->", torch.linalg.matrix_norm(electronic_repulsion_tensor - electronic_repulsion_tensor.transpose(1, 2)))

tensor(216.4966, dtype=torch.float64)

j<->l, not OK

In [18]:
torch.einsum("ij->", torch.linalg.matrix_norm(electronic_repulsion_tensor - electronic_repulsion_tensor.transpose(1, 3)))

tensor(216.4966, dtype=torch.float64)

k<->l, OK

In [19]:
torch.einsum("ij->", torch.linalg.matrix_norm(electronic_repulsion_tensor - electronic_repulsion_tensor.transpose(2, 3)))

tensor(6.8773e-14, dtype=torch.float64)

Let's sum our two tensors (NB: now it loses symmetry of XC-tensor):

In [20]:
fourDtensor = dvxc_wrt_dro + electronic_repulsion_tensor
fourDtensor.size()

torch.Size([18, 18, 18, 18])

Let's calculate the second term:
$$\frac{\partial r_{ia}(\textbf{C};\;\vec{\theta})}{\partial C_{ck}} = (F_{ik}[\rho](\vec{\theta}) - \epsilon_c\delta_{ik})\delta_{ac} + 2f_c\sum_j \sum_{l}C_{cl}\left(G_{Cou,ijkl} + G_{XC,ijkl}\right)C_{aj}$$

In [21]:
from typing import Union
import xitorch as xt
from dqc.utils.datastruct import SpinParam

def _symm(scp: torch.Tensor):
    # forcely symmetrize the tensor
    return (scp + scp.transpose(-2, -1)) * 0.5

def scp2orbital_energies(hf_engine, scp: torch.Tensor) -> Union[torch.Tensor, SpinParam[torch.Tensor]]:
    # scp is like KS, using the concatenated Fock matrix
    if not hf_engine._polarized:
        fock = xt.LinearOperator.m(_symm(scp), is_hermitian=True)
        return fock2orbital_energies(hf_engine, fock)
    else:
        fock_u = xt.LinearOperator.m(_symm(scp[0]), is_hermitian=True)
        fock_d = xt.LinearOperator.m(_symm(scp[1]), is_hermitian=True)
        return fock2orbital_energies(hf_engine, SpinParam(u=fock_u, d=fock_d))

def fock2orbital_energies(hf_engine, fock):
    # diagonalize the fock matrix and obtain the density matrix
    eigvals, _ = hf_engine.diagonalize(fock, hf_engine._norb)
    return eigvals

def scp2orbital_decomposition(hf_engine, scp: torch.Tensor) -> Union[torch.Tensor, SpinParam[torch.Tensor]]:
    # scp is like KS, using the concatenated Fock matrix
    if not hf_engine._polarized:
        fock = xt.LinearOperator.m(_symm(scp), is_hermitian=True)
        return fock2orbital_decomposition(hf_engine, fock)
    else:
        fock_u = xt.LinearOperator.m(_symm(scp[0]), is_hermitian=True)
        fock_d = xt.LinearOperator.m(_symm(scp[1]), is_hermitian=True)
        return fock2orbital_decomposition(hf_engine, SpinParam(u=fock_u, d=fock_d))

def fock2orbital_decomposition(hf_engine, fock):
    _, eigvectors = hf_engine.diagonalize(fock, hf_engine._norb)
    return eigvectors

In [22]:
fock = qc._engine.dm2scp(dm).clone()
coeff = scp2orbital_decomposition(qc._engine.hf_engine, fock)
orbital_energies = scp2orbital_energies(qc._engine.hf_engine, fock)

In [29]:
coeff.size()

torch.Size([18, 7])

In [31]:
dr_wrt_dC = torch.einsum("ijkl,lc,ja->iack", fourDtensor, coeff, coeff)

In [32]:
dr_wrt_dC.size()

torch.Size([18, 7, 7, 18])